In [1]:
import os
from PIL import Image, ImageEnhance, ImageFilter
import pandas as pd
from tqdm import tqdm
import random
import numpy as np

In [2]:
# Define paths
csv_file = 'train.csv'  
img_dir = 'train'  
output_img_dir = 'augmented_images'  # Folder to save augmented images
output_csv_file = 'augmented_train.csv'  # CSV file to save updated data

In [3]:
# Load csv
train_df = pd.read_csv(csv_file)

In [4]:
print("Initial DataFrame:")
print(train_df.head())
print("\nDataFrame Info:")
print(train_df.info())

Initial DataFrame:
                                 Id  Subject Focus  Eyes  Face  Near  Action  \
0  0007de18844b0dbbb5e1f607da0606e0              0     1     1     1       0   
1  0009c66b9439883ba2750fb825e1d7db              0     1     1     0       0   
2  0013fd999caf9a3efe1352ca1b0d937e              0     1     1     1       0   
3  0018df346ac9c1d8413cfcc888ca8246              0     1     1     1       0   
4  001dc955e10590d3ca4673f034feeef2              0     0     0     1       0   

   Accessory  Group  Collage  Human  Occlusion  Info  Blur  Pawpularity  
0          0      1        0      0          0     0     0           63  
1          0      0        0      0          0     0     0           42  
2          0      0        0      1          1     0     0           28  
3          0      0        0      0          0     0     0           15  
4          0      1        0      0          0     0     0           72  

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
R

# CSV

Preprocessing

In [5]:
# Checking for any missing values
if train_df.isnull().any().any():
    print("Missing values found:")
    print(train_df.isnull().sum())
    
else:
    print("No missing values")

No missing values


In [6]:
# Checking for duplicates
duplicates = train_df.duplicated().sum()
if duplicates > 0:
    print(f"Duplicates found: {duplicates}")
    train_df.drop_duplicates(inplace=True)
    
else:
    print("No duplicates found")

No duplicates found


In [7]:
# Converting pawpularity to float
train_df['Pawpularity'] = train_df['Pawpularity'].astype(float)

# Normalize Pawpularity to a range between 0 and 1
train_df['Pawpularity'] = train_df['Pawpularity'] / 100.0  # [0, 1]


# Images

Augmentation Functions

In [8]:
def resize_image(img, size=(224, 224)):
    return img.resize(size)

def normalize_image(img):
    img_array = np.array(img) / 255.0
    return Image.fromarray((img_array * 255).astype(np.uint8))  # Convert back to image if needed

def random_flip(img):
    if random.random() > 0.5:
        return img.transpose(Image.FLIP_LEFT_RIGHT)
    return img

def random_vertical_flip(img):
    if random.random() > 0.5:
        return img.transpose(Image.FLIP_TOP_BOTTOM)
    return img

def random_rotation(img, max_angle=10): 
    angle = random.uniform(-max_angle, max_angle)
    return img.rotate(angle)

def color_jitter(img, brightness=0.1, contrast=0.1, saturation=0.1):  
    img = ImageEnhance.Brightness(img).enhance(1 + random.uniform(-brightness, brightness))
    img = ImageEnhance.Contrast(img).enhance(1 + random.uniform(-contrast, contrast))
    img = ImageEnhance.Color(img).enhance(1 + random.uniform(-saturation, saturation))
    return img

def gaussian_blur(img, radius=0.5): 
    return img.filter(ImageFilter.GaussianBlur(radius))

def random_grayscale(img, p=0.1): 
    if random.random() < p:
        return img.convert("L").convert("RGB")  # Convert to grayscale and back to RGB
    return img

def random_noise(img, noise_factor=0.01):  
    img_array = np.array(img)
    noise = np.random.normal(0, noise_factor * 255, img_array.shape).astype(np.uint8)
    noisy_img_array = np.clip(img_array + noise, 0, 255)
    return Image.fromarray(noisy_img_array.astype(np.uint8))

def random_perspective(img, distortion=0.1): 
    width, height = img.size
    x_shift = distortion * width
    y_shift = distortion * height

    coeffs = [
        x_shift, y_shift,
        width - x_shift, y_shift,
        width - x_shift, height - y_shift,
        x_shift, height - y_shift,
    ]
    return img.transform(img.size, Image.QUAD, coeffs)


In [9]:
# List to hold rows for the new CSV file
augmented_rows = []

Augmenting images

In [10]:
def augment_images(train_df, img_dir, output_img_dir):
    if not os.path.exists(output_img_dir):
        os.makedirs(output_img_dir)

    for idx, row in tqdm(train_df.iterrows(), total=len(train_df)):
        original_id = row['Id']
        pawpularity_score = row['Pawpularity']
        # Include additional columns to copy over
        subject_focus = row['Subject Focus']
        eyes = row['Eyes']
        face = row['Face']
        near = row['Near']
        action = row['Action']
        accessory = row['Accessory']
        group = row['Group']
        collage = row['Collage']
        human = row['Human']
        occlusion = row['Occlusion']
        info = row['Info']
        blur = row['Blur']

        img_path = os.path.join(img_dir, f"{original_id}.jpg")

        # Append the original row (for the unaugmented image)
        augmented_rows.append(row.to_dict())

        # Open and transform the image
        try:
            with Image.open(img_path) as img:
                # Preprocessing: Resize and normalize
                original_resized = resize_image(img)
                original_resized = normalize_image(original_resized)
                original_resized.save(os.path.join(output_img_dir, f"{original_id}.jpg"))

                # Augment and save
                for i in range(5):  # Number of augmented versions per image
                    augmented_img = resize_image(img)
                    augmented_img = normalize_image(augmented_img)
                    augmented_img = random_flip(augmented_img)
                    augmented_img = random_vertical_flip(augmented_img)
                    augmented_img = random_rotation(augmented_img)
                    augmented_img = color_jitter(augmented_img)
                    augmented_img = gaussian_blur(augmented_img)
                    augmented_img = random_grayscale(augmented_img)
                    augmented_img = random_noise(augmented_img)
                    augmented_img = random_perspective(augmented_img)

                    # Save the augmented image with a unique filename
                    augmented_id = f"{original_id}_aug{i}"
                    augmented_img.save(os.path.join(output_img_dir, f"{augmented_id}.jpg"))

                    # Add a row for the augmented image in the new CSV data
                    augmented_rows.append({
                        'Id': augmented_id,
                        'Pawpularity': pawpularity_score,
                        'Subject Focus': subject_focus,
                        'Eyes': eyes,
                        'Face': face,
                        'Near': near,
                        'Action': action,
                        'Accessory': accessory,
                        'Group': group,
                        'Collage': collage,
                        'Human': human,
                        'Occlusion': occlusion,
                        'Info': info,
                        'Blur': blur,
                    })
        except Exception as e:
            print(f"Error processing image {img_path}: {e}")

# Call the function to perform augmentations
augment_images(train_df, img_dir, output_img_dir)

100%|██████████| 9912/9912 [24:40<00:00,  6.69it/s]


In [11]:
# Combine augmented data with preprocessed CSV
if augmented_rows:
    # Create a DataFrame from the augmented rows
    augmented_df = pd.DataFrame(augmented_rows)
    
    # Load existing preprocessed CSV
    if os.path.exists(output_csv_file):
        existing_df = pd.read_csv(output_csv_file)
        
        # Combine the two DataFrames
        combined_df = pd.concat([existing_df, augmented_df], ignore_index=True)
    else:
        # If the CSV does not exist, use only the augmented DataFrame
        combined_df = augmented_df

    # Save the combined data back to the preprocessed CSV file
    combined_df.to_csv(output_csv_file, index=False)
    print("Combined data saved to:", output_csv_file)
else:
    print("No augmented data to combine.")




Combined data saved to: augmented_train.csv
